Install required libraries

In [ ]:
# Install required libraries
!pip install -q google-generativeai matplotlib scikit-learn seaborn pandas
!pip install google-cloud-storage

Use below if fail to get datset from cloud storage

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving Intent_detection_Data.xlsx to Intent_detection_Data.xlsx


In [ ]:
# Load the dataset
file_path = 'Intent_detection_Data.xlsx'

try:
    df = pd.read_excel(file_path)
    print("File loaded successfully.")
except FileNotFoundError:
    print("Error: Intent_detection_Data.xlsx not found in the Colab folder.")
    raise

File loaded successfully.


Get Dataset from Google Cloud Storage

In [ ]:
from google.cloud import storage
import pandas as pd

# Initialize the Google Cloud Storage client
client = storage.Client()

# Set the name of your bucket and file
bucket_name = 'cip_dataset_bucket'
file_name = 'Intent_detection_Data.xlsx'

try:
  # Get the bucket and blob (file)
  bucket = client.get_bucket(bucket_name)
  blob = bucket.blob(file_name)
  # Read the file content as a byte stream
  file_content = blob.download_as_bytes()
  df = pd.read_excel(io.BytesIO(file_content))
except FileNotFoundError:
    print("Error: Intent_detection_Data.xlsx not found.")
    exit()
print("File loaded successfully.")
print(f"Intents-> {df['INTENT'].unique()}")
print(df['INTENT'].value_counts())
print(df.head())

RefreshError: ("Failed to retrieve http://metadata.google.internal/computeMetadata/v1/instance/service-accounts/default/?recursive=true from the Google Compute Engine metadata service. Status: 404 Response:\nb''", <google.auth.transport.requests._Response object at 0x7f9a006c09a0>)

In [ ]:
import io
from google.colab import userdata
import google.generativeai as genai

# Configure the API key for Google Generative AI
genai.configure(api_key=userdata.get('GOOGLE_API_KEY'))

Categories to classify

In [ ]:
# Predefined intent categories
intent_categories = ["CatalogSelection", "OrderStatus", "ProductReplacement", "PriceAndAvailability"]

The prepare_data function prepares data for intent classification by:

splitting the dataset into training, evaluation, and testing sets.
formatting the data for a large language model with customer requests and their corresponding intent.
providing the true intent labels for evaluating the model's accuracy.
Essentially, it transforms raw data into a format ready for training and evaluating a machine learning model for intent classification.

In [ ]:
def prepare_data(df, train_size=0.75, eval_size=0.1, random_state=95):
    # Shuffle the dataset
    df = df.sample(frac=1, random_state=random_state).reset_index(drop=True)

    # Calculate split indices
    train_end = int(train_size * len(df))
    eval_end = train_end + int(eval_size * len(df))

    # Split the dataset
    X_train = df.iloc[:train_end].copy()  # Training data
    X_eval = df.iloc[train_end:eval_end].copy()  # Evaluation data
    X_test = df.iloc[eval_end:].copy()  # Test data

   print( X_train.INTENT.value_counts())
   print( X_eval.INTENT.value_counts())
   print( X_test.INTENT.value_counts())


    # Validate required columns
    required_columns = ['REQUEST', 'INTENT']
    for column in required_columns:
        if column not in df.columns:
            raise ValueError(f"Missing required column: '{column}' in the DataFrame.")

    # Prepare structured training data
    structured_train_data = [
    {
        "text_input": f"""Classify the customer request into one of these predefined categories:
        - PriceAndAvailability: Enquiries about pricing, stock availability, or lead times.
        - ProductReplacement: Enquiries about alternatives, replacements, or discontinued items.
        - OrderStatus: Enquiries about delivery status, tracking, purchase orders (PO), or estimated time of arrival (ETA).
        - CatalogSelection: Enquiries about product specifications, models, or features.

        Please return only one of the four predefined intents exactly as specified without any extra text or symbols.\n
        Request: {row["REQUEST"]}""", "output": row["INTENT"]}
    for _, row in X_train.iterrows()
    ]
    # Prepare structured evaluation data
    structured_eval_data = [
      {
        "text_input": f"""Classify the customer request into one of these predefined categories:
        - PriceAndAvailability: Enquiries about pricing, stock availability, or lead times.
        - ProductReplacement: Enquiries about alternatives, replacements, or discontinued items.
        - OrderStatus: Enquiries about delivery status, tracking, purchase orders (PO), or estimated time of arrival (ETA).
        - CatalogSelection: Enquiries about product specifications, models, or features.

        Please return only one of the four predefined intents exactly as specified without any extra text or symbols.\n
        Request: {row["REQUEST"]}""", "output": row["INTENT"]}
    for _, row in X_eval.iterrows()
    ]

    # Prepare test prompts for inference
    test_prompts = [
        f"""Classify the customer request into one of these categories: {', '.join(intent_categories)}
        - PriceAndAvailability: enquiry about pricing, stock availability, or lead times.
        - ProductReplacement: enquiry about alternatives, replacements, or discontinued items.
        - OrderStatus: enquiry about delivery status, tracking, purchase orders (PO), or estimated time of arrival (ETA).
        - CatalogSelection: enquiry about product specifications, models, or features.

        Request: {row["REQUEST"]}
        output:"""
        for _, row in X_test.iterrows()
    ]

    # Extract true labels for evaluation
    y_true = X_test['INTENT'].tolist()

    return {
        "structured_train_data": structured_train_data,
        "structured_eval_data": structured_eval_data,
        "test_prompts": test_prompts,
        "y_true": y_true
    }


The code cleans and prepares a dataset for intent classification.
It splits the data into training, evaluation, and testing sets, formatting it for a large language model.
The prepared data is then stored in variables for use in training and evaluating the model.

In [ ]:
# Prepare the data
df['INTENT'] = df['INTENT'].replace("Product Replacement", "ProductReplacement")
data_splits = prepare_data(df)
# Access the outputs
structured_train_data = data_splits["structured_train_data"]
structured_eval_data = data_splits["structured_eval_data"]
test_prompts = data_splits["test_prompts"]
y_true = data_splits["y_true"]

['ProductReplacement' 'OrderStatus' 'PriceAndAvailability'
 'CatalogSelection']
['CatalogSelection' 'ProductReplacement' 'OrderStatus'
 'PriceAndAvailability']
['PriceAndAvailability' 'ProductReplacement' 'CatalogSelection'
 'OrderStatus']


Fine-tuning a Google Gemini model, providing you with a specialized model ready for your intent classification task.

In [ ]:
# Gemini model to tune
base_model = "models/gemini-1.5-flash-001-tuning"
name = "GeminiCustomerIntentPredictionModel"

In [ ]:
def fine_tune_gemini(base_model, name, training_data, batch_size, learning_rate, epoch_count):
    global fine_tuned_model
    try:
        print(f"Starting fine-tuning for model: {base_model} with display name: {name}")
        operation = genai.create_tuned_model(
            display_name=name,
            source_model=base_model,
            epoch_count=epoch_count,
            batch_size=batch_size,
            learning_rate=learning_rate,
            training_data=training_data  # Use the provided argument
        )

        print("Fine-tuning in progress. Please wait...")
        while not operation.done():
            print("Training in progress... Checking status...")
            time.sleep(5)

        # Access the fine-tuned model details
        result = operation.result()
        fine_tuned_model = result.name  # Assign to the global variable
        print("Model trained successfully!")
        print(f"Trained Model Details:\n{result}")
    except Exception as e:
        print(f"An error occurred during fine-tuning: {e}")
        raise


Start tuning---

In [ ]:
#fine_tune_gemini(base_model, name, structured_train_data, batch_size=8, learning_rate=0.0001, epoch_count=10)--81% split 70%
#fine_tune_gemini(base_model, name, structured_train_data, batch_size=7, learning_rate=0.0002, epoch_count=9) --81% - split 80%
fine_tune_gemini(base_model, name, structured_train_data, batch_size=6, learning_rate=0.0001, epoch_count=8) #--split 75%

Starting fine-tuning for model: models/gemini-1.5-flash-001-tuning with display name: GeminiCustomerIntentPredictionModel
Fine-tuning in progress. Please wait...
Training in progress... Checking status...
Training in progress... Checking status...
Training in progress... Checking status...
Training in progress... Checking status...
Training in progress... Checking status...
Training in progress... Checking status...
Training in progress... Checking status...
Training in progress... Checking status...
Training in progress... Checking status...
Training in progress... Checking status...
Training in progress... Checking status...
Training in progress... Checking status...
Training in progress... Checking status...
Training in progress... Checking status...
Training in progress... Checking status...
Training in progress... Checking status...
Training in progress... Checking status...
Training in progress... Checking status...
Training in progress... Checking status...
Training in progress.

In [ ]:
print(fine_tuned_model)

tunedModels/geminicustomerintentpredictionmodel-q44w


In [ ]:
import time
# Ensure generation configuration is properly applied
generation_config = {
    "temperature": 0.7,  # Lower for less randomness
    "top_p": 0.6,        # Balanced nucleus sampling
    "top_k": 20,         # Less diversity for predictability
    "max_output_tokens": 50,  # Ensure concise outputs
    "response_mime_type": "text/plain",
}

# Create zero-shot (base) model
base_model = genai.GenerativeModel(
    model_name="gemini-1.5-flash",
    generation_config=generation_config,
)

# Create fine-tuned model instance
fine_tuned_model_instance = genai.GenerativeModel(
    model_name=fine_tuned_model,  # Use the correct fine-tuned model name
    generation_config=generation_config,
)

# Manual validation for both zero-shot and fine-tuned predictions
zero_shot_predictions = []
few_shot_predictions = []

for example in structured_eval_data:
    # Zero-shot predictions
    response = base_model.generate_content(example["text_input"])
    zero_shot_predictions.append(response.text.strip())  # Use response.text.strip() as required
    time.sleep(2)
    # Few-shot predictions (fine-tuned)
    response = fine_tuned_model_instance.generate_content(example["text_input"])
    few_shot_predictions.append(response.text.strip())  # Use response.text.strip() as required

In [ ]:
from sklearn.metrics import classification_report,accuracy_score, precision_recall_fscore_support, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

# Extract true labels
true_labels = [example["output"] for example in structured_eval_data]

# Compute evaluation metrics for zero-shot predictions
print("\nZero-Shot Predictions Metrics:")
zero_shot_accuracy = accuracy_score(true_labels, zero_shot_predictions)
print(f"Zero-Shot Validation Accuracy: {zero_shot_accuracy:.2f}")
print("\nZero-Shot Classification Report:")
print(classification_report(true_labels, zero_shot_predictions, zero_division=0))

# Compute evaluation metrics for few-shot predictions
print("\nFew-Shot Predictions Metrics (Fine-Tuned Model):")
few_shot_accuracy = accuracy_score(true_labels, few_shot_predictions)
print(f"Few-Shot Validation Accuracy: {few_shot_accuracy:.2f}")
print("\nFew-Shot Classification Report:")
print(classification_report(true_labels, few_shot_predictions, zero_division=0))



Zero-Shot Predictions Metrics:
Zero-Shot Validation Accuracy: 0.88

Zero-Shot Classification Report:
                      precision    recall  f1-score   support

    CatalogSelection       1.00      0.86      0.92         7
         OrderStatus       1.00      1.00      1.00         3
PriceAndAvailability       0.60      1.00      0.75         3
  ProductReplacement       1.00      0.67      0.80         3

            accuracy                           0.88        16
           macro avg       0.90      0.88      0.87        16
        weighted avg       0.93      0.88      0.88        16


Few-Shot Predictions Metrics (Fine-Tuned Model):
Few-Shot Validation Accuracy: 0.94

Few-Shot Classification Report:
                      precision    recall  f1-score   support

    CatalogSelection       1.00      0.86      0.92         7
         OrderStatus       1.00      1.00      1.00         3
PriceAndAvailability       0.75      1.00      0.86         3
  ProductReplacement       1.00 